In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib as mpl
from gamap_colormap import WhGrYlRd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from ipywidgets import interact, IntSlider, SelectionSlider, Dropdown
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
file = "SpeciesConc"
normal_file = f"/home/ucfacb0/rundirs/Rockets/gc_4x5_47L_merra2_fullchem_aluminachemtesting/OutputDir/norrtmg/normalchem_aermassout/GEOSChem.{file}.20200101_0000z.nc4"
gravsettle_file = f"/home/ucfacb0/rundirs/Rockets/gc_4x5_47L_merra2_fullchem_aluminachemtesting/OutputDir/norrtmg/gravsettlechem3/GEOSChem.{file}.20200101_0000z.nc4"
ds_gravsettle = xr.open_dataset(gravsettle_file)
ds_normal = xr.open_dataset(normal_file)
ds_normal

<xarray.Dataset>
Dimensions:                 (time: 1, lev: 47, ilev: 48, lat: 46, nb: 2, lon: 72)
Coordinates:
  * time                    (time) datetime64[ns] 2020-01-01
  * lev                     (lev) float64 0.9925 0.9775 ... 0.0001387 3.8e-05
  * ilev                    (ilev) float64 1.0 0.985 0.97 ... 6.6e-05 1e-05
  * lat                     (lat) float64 -89.0 -86.0 -82.0 ... 82.0 86.0 89.0
  * lon                     (lon) float64 -180.0 -175.0 -170.0 ... 170.0 175.0
Dimensions without coordinates: nb
Data variables: (12/319)
    lat_bnds                (lat, nb) float64 ...
    lon_bnds                (lon, nb) float64 ...
    hyam                    (lev) float64 ...
    hybm                    (lev) float64 ...
    hyai                    (ilev) float64 ...
    hybi                    (ilev) float64 ...
    ...                      ...
    SpeciesConcVV_AONITA    (time, lev, lat, lon) float32 ...
    SpeciesConcVV_ALK4      (time, lev, lat, lon) float32 ...
    SpeciesConcVV_ALD2      (time, lev, lat, lon) float32 ...
    SpeciesConcVV_AERI      (time, lev, lat, lon) float32 ...
    SpeciesConcVV_ACTA      (time, lev, lat, lon) float32 ...
    SpeciesConcVV_ACET      (time, lev, lat, lon) float32 ...
Attributes:
    title:                           GEOS-Chem diagnostic collection: Species...
    history:                         
    format:                          NetCDF-4
    conventions:                     COARDS
    ProdDateTime:                    
    reference:                       www.geos-chem.org; wiki.geos-chem.org
    contact:                         GEOS-Chem Support Team (geos-chem-suppor...
    simulation_start_date_and_time:  2020-01-01 00:00:00z
    simulation_end_date_and_time:    2020-01-02 00:00:00z

In [2]:
# List of species we want
varnames = ["SpeciesConcVV_DST1",
            "SpeciesConcVV_DST2",
            "SpeciesConcVV_DST3",
            "SpeciesConcVV_DST4",
            "SpeciesConcVV_BCPO",
            "SpeciesConcVV_BCPI",
            "SpeciesConcVV_SO4",
            "SpeciesConcVV_O3"]

def plot_layer(var, l):
    fig = plt.figure(figsize=[8,4])
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines()
    ax.gridlines(linestyle='--')
    species_range_normal = ds_normal[var].isel(time=0,lev=l-1)
    species_range_gravsettle = ds_gravsettle[var].isel(time=0,lev=l-1)
    species_range = species_range_gravsettle-species_range_normal
    max_range = np.max(species_range)
    species_range.plot.pcolormesh(ax=ax, cmap=WhGrYlRd,vmin=-max_range,vmax=max_range)
    ax.set_title(var+f'; lev={l}')

interact(plot_layer, 
         var=SelectionSlider(options=varnames, continuous_update=True), 
         l=IntSlider(min=1, max=46, step=1, continuous_update=True),
        );


interactive(children=(SelectionSlider(description='var', options=('SpeciesConcVV_DST1', 'SpeciesConcVV_DST2', …

In [3]:
# PLot zonal mean over lon for all species we want.

# Should plot stratopause.
import netCDF4
array_4d_tl = np.zeros((46,72))
met = netCDF4.Dataset('/home/ucfacb0/Scratch/StateMet/4x5/2020/GEOSChem.StateMet.20200101_0000z.nc4')
array_4d_tl[:,:] = met.variables['Met_TropP'][0,:,:] # tropopause height grid level
average_tl = np.mean(array_4d_tl,axis=1) 
        
Plower = 1000
Pupper = 10
time, lon, lat = ds_normal.variables['time'][:], ds_normal.variables['lon'][:], ds_normal.variables['lat'][:]
P0, hyai, hybi = ds_normal.variables['P0'], ds_normal.variables['hyai'][:], ds_normal.variables['hybi'][:]
Pedge = [ hyai[i] + (hybi[i]*1000) for i in np.arange(len(hyai)) ]
iu = np.argmin(np.abs([ Pedge[i] - Pupper for i in np.arange(len(Pedge)) ] )) # grid index of upper pressure level
il = np.argmin(np.abs([ Pedge[i] - Plower for i in np.arange(len(Pedge)) ] )) # grid index of lower pressure level   
Pedge = Pedge[il:iu]

def plot_layer(var):
    fig, ax = plt.subplots()
    species_range_normal = ds_normal[var].isel(time=0,lev=(np.arange(il,iu))).mean(dim='lon')
    species_range_gravsettle = ds_gravsettle[var].isel(time=0,lev=(np.arange(il,iu))).mean(dim='lon')
    species_range = species_range_gravsettle-species_range_normal
    max_range = np.max(species_range)
    cf = ax.contourf(lat, Pedge, species_range, cmap=WhGrYlRd, levels=np.linspace(-max_range, max_range, 21))
    ax.plot(lat,average_tl,'-.',color="black")
    ax.set_yscale('log')
    ax.invert_yaxis()
    ax.set_xlabel("Latitude ($^{\circ}$)")
    ax.set_ylabel("Pressure (hPa)")
    cbar = fig.colorbar(cf, ax=ax, orientation='vertical', ticks=np.linspace(-max_range,max_range,num=5))
    ax.set_title(var)

interact(plot_layer, 
         var=SelectionSlider(options=varnames, continuous_update=True), 
        );

interactive(children=(SelectionSlider(description='var', options=('SpeciesConcVV_DST1', 'SpeciesConcVV_DST2', …

In [4]:
ds_gravsettle.close()
ds_normal.close()